In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras 

# https://rfriend.tistory.com/431
import os # miscellaneous operating system interfaces
import shutil # high-level file operations

base_dir = '/Users/82105/Desktop/bigdata_analysis/data'
img_dir = 'C:/Users/82105/Desktop/bigdata_analysis/data/dogs-vs-cats/test1/test1'
len(os.listdir(img_dir))

In [ ]:
# 30개 고양이 사진 옮기기
cats30_dir = os.path.join(base_dir,'dog_test')

# 디렉토리만들기
os.mkdir(cats30_dir)

In [ ]:
# 이미지 복사 30장만
fnames = ['dog.{}.jpg'.format(i) for i in range(3000)]

for fname in fnames:
    src = os.path.join(img_dir, fname)
    dst = os.path.join(cats30_dir, fname)
    shutil.copyfile(src,dst)

In [1]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

caltech_dir = "C:/Users/82105/Desktop/bigdata_analysis/data/train"
categories = ["dog", "cat"]
nb_classes = len(categories)

image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

        if i % 700 == 0:
            print(cat, " : ", f)

X = np.array(X)
y = np.array(y)
#1 0 0 0 이면 airplanes
#0 1 0 0 이면 buddha 이런식


X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("./multi_image_data.npy", xy)

print("ok", len(y))

dog  파일 길이 :  10000
dog  :  C:/Users/82105/Desktop/bigdata_analysis/data/train/dog\dog.0.jpg
dog  :  C:/Users/82105/Desktop/bigdata_analysis/data/train/dog\dog.1628.jpg
dog  :  C:/Users/82105/Desktop/bigdata_analysis/data/train/dog\dog.2258.jpg
dog  :  C:/Users/82105/Desktop/bigdata_analysis/data/train/dog\dog.2889.jpg
dog  :  C:/Users/82105/Desktop/bigdata_analysis/data/train/dog\dog.3518.jpg
dog  :  C:/Users/82105/Desktop/bigdata_analysis/data/train/dog\dog.4148.jpg
dog  :  C:/Users/82105/Desktop/bigdata_analysis/data/train/dog\dog.4779.jpg
dog  :  C:/Users/82105/Desktop/bigdata_analysis/data/train/dog\dog.5408.jpg
dog  :  C:/Users/82105/Desktop/bigdata_analysis/data/train/dog\dog.6038.jpg
dog  :  C:/Users/82105/Desktop/bigdata_analysis/data/train/dog\dog.6669.jpg
dog  :  C:/Users/82105/Desktop/bigdata_analysis/data/train/dog\dog.7299.jpg
dog  :  C:/Users/82105/Desktop/bigdata_analysis/data/train/dog\dog.7929.jpg
dog  :  C:/Users/82105/Desktop/bigdata_analysis/data/train/dog\dog.8559

FileNotFoundError: [Errno 2] No such file or directory: './numpy_data/multi_image_data.npy'

In [2]:
np.save("./multi_image_data.npy", xy)

In [6]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

X_train, X_test, y_train, y_test = np.load('./multi_image_data.npy',allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])

(15000, 64, 64, 3)
15000


In [7]:

categories = ["dog", "cat"]
nb_classes = len(categories)

#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

In [8]:
with K.tf_ops.device('/device:GPU:0'):
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_dir = './model'
    
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    
    model_path = model_dir + '/multi_img_classification.model'
    checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16384)            

In [10]:
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])


Train on 15000 samples, validate on 5000 samples
Epoch 1/50
15000/15000 [==============================] - 80s 5ms/step - loss: 0.6680 - accuracy: 0.6065 - val_loss: 0.5802 - val_accuracy: 0.7078

Epoch 00001: val_loss improved from inf to 0.58023, saving model to ./model/multi_img_classification.model
Epoch 2/50
15000/15000 [==============================] - 74s 5ms/step - loss: 0.5728 - accuracy: 0.7015 - val_loss: 0.5743 - val_accuracy: 0.7134

Epoch 00002: val_loss improved from 0.58023 to 0.57430, saving model to ./model/multi_img_classification.model
Epoch 3/50
15000/15000 [==============================] - 77s 5ms/step - loss: 0.5204 - accuracy: 0.7429 - val_loss: 0.5049 - val_accuracy: 0.7584

Epoch 00003: val_loss improved from 0.57430 to 0.50488, saving model to ./model/multi_img_classification.model
Epoch 4/50
15000/15000 [==============================] - 76s 5ms/step - loss: 0.4819 - accuracy: 0.7651 - val_loss: 0.4851 - val_accuracy: 0.7740

Epoch 00004: val_loss improve

KeyboardInterrupt: 

In [ ]:
print("정확도 : %.4f" % (model.evaluate(X_test, y_test)[1]))


In [ ]:
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c='red', label='val_set_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='train_set_oss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.show()

In [ ]:
from PIL import Image
import os, glob, numpy as np
from keras.models import load_model

caltech_dir = "C:/Users/82105/Desktop/bigdata_analysis/data/dogs-vs-cats/tt"
image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
filenames = []
files = glob.glob(caltech_dir+"/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)
model = load_model('./multi_img_classification.model')

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt = 0

#이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    print(i)
    print(pre_ans)
    pre_ans_str = ''
    if pre_ans == 0: pre_ans_str = "강아지"
    else: pre_ans_str = "고양이"
    if i[0] >= 0.8 : print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[1] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"으로 추정됩니다.")
    cnt += 1
    # print(i.argmax()) #얘가 레이블 [1. 0. 0.] 이런식으로 되어 있는 것을 숫자로 바꿔주는 것.
    # 즉 얘랑, 나중에 카테고리 데이터 불러와서 카테고리랑 비교를 해서 같으면 맞는거고, 아니면 틀린거로 취급하면 된다.
    # 이걸 한 것은 _4.py에.